# GlobeStay Travel — Data Cleaning and Descriptive Analysis

This notebook focuses on two scopes only:

- Data cleaning: load, standardize, parse dates/weeks, normalize country labels, coerce numerics, basic outlier handling
- Descriptive analysis: summaries by country/channel and time-series visualizations

Inputs
- `data_mmm_2020_post.xlsx` (weekly panel for US, UK, Germany)

Outputs
- Cleaned dataset saved to `outputs/`
- Summary tables and charts saved to `outputs/`


In [21]:
# Imports and configuration
import os
import re
from pathlib import Path
from typing import List

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 200)
sns.set_theme(style="whitegrid")

PROJECT_ROOT = Path.cwd()
DATA_FILENAME_CANDIDATES: List[str] = [
    "data_mmm_2020_post.xlsx",
]
OUTPUTS_DIR = PROJECT_ROOT / "outputs"
OUTPUTS_DIR.mkdir(exist_ok=True)

# Utility logging
from datetime import datetime

def log(msg: str) -> None:
    ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{ts}] {msg}")

# Resolve data path
excel_path: Path | None = None
for candidate in DATA_FILENAME_CANDIDATES:
    p = PROJECT_ROOT / candidate
    if p.exists():
        excel_path = p
        break

if excel_path is None:
    # Fallback: pick the first .xlsx in root
    xlsx_files = list(PROJECT_ROOT.glob("*.xlsx"))
    excel_path = xlsx_files[0] if xlsx_files else None

log(f"Detected Excel file: {excel_path}")



[2025-10-12 10:36:14] Detected Excel file: c:\Users\Administrator.等闲的电脑\Downloads\831\data_mmm_2020_post.xlsx


In [22]:
# Ensure inline plots in classic Notebook
%matplotlib inline


In [23]:
# Load Excel: list sheets and preview
if excel_path is None:
    raise FileNotFoundError("No Excel file found in project root. Place data_mmm_2020_post.xlsx here.")

xlsx = pd.ExcelFile(excel_path)
log(f"Sheets found: {xlsx.sheet_names}")

sheets_to_load = xlsx.sheet_names
frames = {}
for sheet in sheets_to_load:
    try:
        df = pd.read_excel(excel_path, sheet_name=sheet, engine="openpyxl")
        frames[sheet] = df
        log(f"Loaded '{sheet}' with shape {df.shape}")
    except Exception as e:
        log(f"WARN: Failed to load sheet '{sheet}': {e}")

# Show quick glimpse of first sheet
first_sheet = sheets_to_load[0] if sheets_to_load else None
if first_sheet:
    display(frames[first_sheet].head(3))



[2025-10-12 10:36:14] Sheets found: ['Sheet1']
[2025-10-12 10:36:14] Loaded 'Sheet1' with shape (591, 67)


,weekstart,country,totbookings,clicks_email,clicks_ppc_brand,clicks_ppc_non_brand,clicks_remarketing,clicks_shop_GoogleHA,clicks_shop_TripAdvisor,clicks_shop_Trivago,clicks_shop_other,cost_email,cost_ppc_brand,cost_ppc_non_brand,cost_remarketing,cost_shop_GoogleHA,cost_shop_TripAdvisor,cost_shop_Trivago,cost_shop_other,display_imps,display_net_spend_eur,olv_imps,olv_net_spend_eur,yt_imps,yt_cost,brandtv_grp,brandtv_net_spend_eur,drtv_grp,drtv_net_spend_eur,ooh_net_spend_eur,radio_net_spend_eur,print_net_spend_eur,cinema_net_spend_eur,meta_comp_grp,ota_comp_grp,value (currency rate),fb_imps,fb_cost,NewYearsDay,MartinL.KingsDay(US-CA),StValentinesDay,PresidentsDay(US-CA),EasterSunday,EasterMonday,LabourDay(DE-NW),MayDay(GB-EN),ChristsAscensionDay(DE-NW),WhitMonday(DE-NW),Remembrance/MemorialDay(US-CA),BankHoliday(GB-EN),IndependenceDay(US-CA),LabourDay(US-CA),GermanUnityDay(DE-NW),ThanksgivingDay(US-CA),ChristmasDay,BoxingDay,sales_Direct,sales_EMK,sales_Interco,sales_ppc_brand,sales_ppc_nonbrand,sales_Retargeting,sales_shop_googleha,sales_shop_other,sales_shop_tripadvisor,sales_shop_trivago,sales_Strat Part
0,2016-01-04,de,398268,474859.0,378640.216496,2.496438e+06,172069.878486,133462.122833,367987.978037,847827.012246,250527.000002,0.0,68560.240287,2.196494e+06,58684.506092,170087.676546,283108.460235,377911.565361,103139.320644,0.0,0.0,0.0,0.0,12.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,926.0,321.0,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,183169,16703,867,48228,87808,2040,10423,4678,9503,18302,16547
1,2016-01-11,de,404068,478024.0,386722.377913,2.520208e+06,198123.396746,136016.480131,353354.844840,961232.479212,278878.649717,0.0,70502.248076,2.306422e+06,76380.207088,193176.386595,257639.565836,471254.197728,116137.878276,0.0,0.0,0.0,0.0,247.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,706.0,225.0,1.0,57831.0,89.386325,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,184480,16278,858,49076,87383,2236,11107,5019,9143,21148,17340
2,2016-01-18,de,388746,482391.0,372694.075197,2.538803e+06,186402.476444,126167.255474,332119.415416,912773.599042,288057.500725,0.0,61647.322276,2.379034e+06,81636.483459,191568.477802,247711.892237,439863.176592,116345.872180,0.0,0.0,0.0,0.0,10249.0,1.459669,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,813.0,231.0,1.0,104754.0,231.027642,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,178895,15822,806,45575,85257,2224,10048,4963,8568,19715,16873


In [24]:
# Standardize column names and concatenate sheets if consistent

def standardize_columns(df: pd.DataFrame) -> pd.DataFrame:
    # lowercase, strip, replace spaces and special chars with underscores
    new_cols = []
    for c in df.columns:
        c2 = str(c).strip().lower()
        c2 = re.sub(r"[^a-z0-9]+", "_", c2)
        c2 = c2.strip("_")
        new_cols.append(c2)
    df.columns = new_cols
    return df

std_frames = {}
for s, d in frames.items():
    std_frames[s] = standardize_columns(d.copy())

# Identify common columns across sheets
sheet_cols = {s: set(df.columns) for s, df in std_frames.items()}
common_cols = set.intersection(*sheet_cols.values()) if std_frames else set()
log(f"Common columns across sheets: {len(common_cols)}")

# If sheets share a schema, concatenate; else keep separate
if len(std_frames) > 0 and len(common_cols) > 3:
    base = pd.concat([df[list(common_cols)].assign(_source_sheet=s) for s, df in std_frames.items()], ignore_index=True)
else:
    # fallback: pick the first standardized sheet
    key = next(iter(std_frames))
    base = std_frames[key].copy()
    base["_source_sheet"] = key

log(f"Base shape after merge/select: {base.shape}")
base.head(3)


[2025-10-12 10:36:14] Common columns across sheets: 67
[2025-10-12 10:36:14] Base shape after merge/select: (591, 68)


,sales_interco,presidentsday_us_ca,sales_direct,cost_email,fb_cost,value_currency_rate,labourday_de_nw,cost_shop_other,sales_shop_other,boxingday,olv_net_spend_eur,cinema_net_spend_eur,sales_retargeting,olv_imps,brandtv_net_spend_eur,print_net_spend_eur,christmasday,newyearsday,cost_ppc_non_brand,sales_shop_trivago,sales_emk,cost_remarketing,clicks_email,drtv_grp,mayday_gb_en,ooh_net_spend_eur,fb_imps,germanunityday_de_nw,cost_shop_tripadvisor,display_net_spend_eur,display_imps,thanksgivingday_us_ca,clicks_shop_googleha,sales_shop_googleha,cost_ppc_brand,totbookings,bankholiday_gb_en,whitmonday_de_nw,clicks_ppc_non_brand,clicks_shop_trivago,cost_shop_googleha,yt_imps,radio_net_spend_eur,labourday_us_ca,ota_comp_grp,drtv_net_spend_eur,weekstart,sales_shop_tripadvisor,christsascensionday_de_nw,clicks_remarketing,cost_shop_trivago,sales_ppc_nonbrand,eastersunday,brandtv_grp,clicks_shop_tripadvisor,stvalentinesday,country,yt_cost,independenceday_us_ca,remembrance_memorialday_us_ca,martinl_kingsday_us_ca,clicks_ppc_brand,meta_comp_grp,eastermonday,sales_strat_part,sales_ppc_brand,clicks_shop_other,_source_sheet
0,867,0.0,183169,0.0,0.000000,1.0,0.0,103139.320644,4678,0.0,0.0,0.0,2040,0.0,0.0,0.0,0.0,0.0,2.196494e+06,18302,16703,58684.506092,474859.0,0.0,0.0,0.0,0.0,0.0,283108.460235,0.0,0.0,0.0,133462.122833,10423,68560.240287,398268,0.0,0.0,2.496438e+06,847827.012246,170087.676546,12.0,0.0,0.0,321.0,0.0,2016-01-04,9503,0.0,172069.878486,377911.565361,87808,0.0,0.0,367987.978037,0.0,de,0.000000,0.0,0.0,0.0,378640.216496,926.0,0.0,16547,48228,250527.000002,Sheet1
1,858,0.0,184480,0.0,89.386325,1.0,0.0,116137.878276,5019,0.0,0.0,0.0,2236,0.0,0.0,0.0,0.0,0.0,2.306422e+06,21148,16278,76380.207088,478024.0,0.0,0.0,0.0,57831.0,0.0,257639.565836,0.0,0.0,0.0,136016.480131,11107,70502.248076,404068,0.0,0.0,2.520208e+06,961232.479212,193176.386595,247.0,0.0,0.0,225.0,0.0,2016-01-11,9143,0.0,198123.396746,471254.197728,87383,0.0,0.0,353354.844840,0.0,de,0.000000,0.0,0.0,0.0,386722.377913,706.0,0.0,17340,49076,278878.649717,Sheet1
2,806,0.0,178895,0.0,231.027642,1.0,0.0,116345.872180,4963,0.0,0.0,0.0,2224,0.0,0.0,0.0,0.0,0.0,2.379034e+06,19715,15822,81636.483459,482391.0,0.0,0.0,0.0,104754.0,0.0,247711.892237,0.0,0.0,0.0,126167.255474,10048,61647.322276,388746,0.0,0.0,2.538803e+06,912773.599042,191568.477802,10249.0,0.0,0.0,231.0,0.0,2016-01-18,8568,0.0,186402.476444,439863.176592,85257,0.0,0.0,332119.415416,0.0,de,1.459669,0.0,0.0,1.0,372694.075197,813.0,0.0,16873,45575,288057.500725,Sheet1


In [25]:
# Basic schema expectations and soft validation
EXPECTED_COUNTRY_ALIASES = {
    "us": ["us", "usa", "united_states"],
    "uk": ["uk", "gb", "united_kingdom", "great_britain"],
    "de": ["de", "ger", "germany", "deutschland"],
}

# Try to detect columns of interest
col_map = {
    "country": None,
    "date": None,  # weekly date or week ending
    "week": None,  # week number if present
    "bookings": None,  # total bookings outcome
}

for c in base.columns:
    if col_map["country"] is None and re.search(r"country|market|geo", c):
        col_map["country"] = c
    if col_map["date"] is None and re.search(r"date|week_end|week_start|week_ending", c):
        col_map["date"] = c
    if col_map["week"] is None and re.search(r"^week$|week_num|week_number", c):
        col_map["week"] = c
    if col_map["bookings"] is None and re.search(r"bookings|orders|sales_total|total_sales", c):
        col_map["bookings"] = c

log(f"Detected columns: {col_map}")

# Identify potential spend/impression/performance columns
spend_cols = [c for c in base.columns if re.search(r"(^|_)spend($|_)|cost|cpc|media_spend", c)]
impr_cols = [c for c in base.columns if re.search(r"impr|impressions", c)]
performance_sales_cols = [c for c in base.columns if re.search(r"^sales_.*|.*_sales$", c)]

log(f"Spend cols: {len(spend_cols)}, Impr cols: {len(impr_cols)}, Perf sales cols: {len(performance_sales_cols)}")



[2025-10-12 10:36:14] Detected columns: {'country': 'cost_remarketing', 'date': None, 'week': None, 'bookings': 'totbookings'}
[2025-10-12 10:36:14] Spend cols: 18, Impr cols: 0, Perf sales cols: 11


In [26]:
# Fallback: create bookings proxy if not detected
if not col_map["bookings"] and performance_sales_cols:
    clean["bookings_proxy"] = clean[performance_sales_cols].sum(axis=1, min_count=1)
    if clean["bookings_proxy"].notna().any():
        col_map["bookings"] = "bookings_proxy"
        log("No explicit 'bookings' detected; using 'bookings_proxy' derived from performance sales.")
    else:
        log("Bookings proxy contained only NaNs; skipping proxy assignment.")


In [27]:
# Cleaning helpers: country normalization, date/week parsing, numeric coercion

COUNTRY_NORMALIZATION = {
    **{alias: "US" for alias in EXPECTED_COUNTRY_ALIASES["us"]},
    **{alias: "UK" for alias in EXPECTED_COUNTRY_ALIASES["uk"]},
    **{alias: "DE" for alias in EXPECTED_COUNTRY_ALIASES["de"]},
}


def normalize_country(val) -> str | None:
    if pd.isna(val):
        return None
    s = str(val).strip().lower()
    return COUNTRY_NORMALIZATION.get(s, str(val).strip())


def parse_weeklike_date(series: pd.Series) -> pd.Series:
    # Try parse as datetime first
    parsed = pd.to_datetime(series, errors="coerce")
    if parsed.notna().mean() > 0.5:
        return parsed.dt.to_period("W").dt.to_timestamp("W-SUN")
    # If parse failed, try integer week numbers with an inferred year (fallback: 2014 as start)
    numeric = pd.to_numeric(series, errors="coerce")
    if numeric.notna().mean() > 0.5:
        # Assume week numbers monotonically increase; build fake dates starting from 2014-01-05 (first Sunday)
        start = pd.Timestamp("2014-01-05")
        return numeric.fillna(0).astype(int).apply(lambda w: start + pd.Timedelta(weeks=max(w-1, 0)))
    return pd.NaT


def coerce_numeric(df: pd.DataFrame, cols: List[str]) -> pd.DataFrame:
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

# Apply cleaning
clean = base.copy()

if col_map["country"]:
    clean["country_std"] = clean[col_map["country"]].map(normalize_country)
else:
    clean["country_std"] = None

if col_map["date"]:
    clean["week_start"] = parse_weeklike_date(clean[col_map["date"]])
elif col_map["week"]:
    clean["week_start"] = parse_weeklike_date(clean[col_map["week"]])
else:
    clean["week_start"] = pd.NaT

numeric_candidates = []
numeric_candidates += spend_cols
numeric_candidates += impr_cols
numeric_candidates += performance_sales_cols
if col_map["bookings"]:
    numeric_candidates.append(col_map["bookings"])

numeric_candidates = sorted(set(numeric_candidates))
clean = coerce_numeric(clean, numeric_candidates)

# Simple outlier capping per column (95th percentile), only for spend/impressions
for c in spend_cols + impr_cols:
    if c in clean.columns:
        q = clean[c].quantile(0.995)
        if pd.notna(q) and q > 0:
            clean[c] = np.clip(clean[c], a_min=None, a_max=q)

# Drop obvious empty rows
before = len(clean)
clean = clean[clean["week_start"].notna() | clean["country_std"].notna()]
after = len(clean)
log(f"Dropped empty rows: {before - after}")

clean.head(3)


[2025-10-12 10:36:14] Dropped empty rows: 15


,sales_interco,presidentsday_us_ca,sales_direct,cost_email,fb_cost,value_currency_rate,labourday_de_nw,cost_shop_other,sales_shop_other,boxingday,olv_net_spend_eur,cinema_net_spend_eur,sales_retargeting,olv_imps,brandtv_net_spend_eur,print_net_spend_eur,christmasday,newyearsday,cost_ppc_non_brand,sales_shop_trivago,sales_emk,cost_remarketing,clicks_email,drtv_grp,mayday_gb_en,ooh_net_spend_eur,fb_imps,germanunityday_de_nw,cost_shop_tripadvisor,display_net_spend_eur,display_imps,thanksgivingday_us_ca,clicks_shop_googleha,sales_shop_googleha,cost_ppc_brand,totbookings,bankholiday_gb_en,whitmonday_de_nw,clicks_ppc_non_brand,clicks_shop_trivago,cost_shop_googleha,yt_imps,radio_net_spend_eur,labourday_us_ca,ota_comp_grp,drtv_net_spend_eur,weekstart,sales_shop_tripadvisor,christsascensionday_de_nw,clicks_remarketing,cost_shop_trivago,sales_ppc_nonbrand,eastersunday,brandtv_grp,clicks_shop_tripadvisor,stvalentinesday,country,yt_cost,independenceday_us_ca,remembrance_memorialday_us_ca,martinl_kingsday_us_ca,clicks_ppc_brand,meta_comp_grp,eastermonday,sales_strat_part,sales_ppc_brand,clicks_shop_other,_source_sheet,country_std,week_start
0,867.0,0.0,183169.0,0.0,0.000000,1.0,0.0,103139.320644,4678.0,0.0,0.0,0.0,2040.0,0.0,0.0,0.0,0.0,0.0,2.196494e+06,18302.0,16703.0,58684.506092,474859.0,0.0,0.0,0.0,0.0,0.0,283108.460235,0.0,0.0,0.0,133462.122833,10423.0,68560.240287,398268.0,0.0,0.0,2.496438e+06,847827.012246,170087.676546,12.0,0.0,0.0,321.0,0.0,2016-01-04,9503.0,0.0,172069.878486,377911.565361,87808.0,0.0,0.0,367987.978037,0.0,de,0.000000,0.0,0.0,0.0,378640.216496,926.0,0.0,16547.0,48228.0,250527.000002,Sheet1,58684.506091931464,NaT
1,858.0,0.0,184480.0,0.0,89.386325,1.0,0.0,116137.878276,5019.0,0.0,0.0,0.0,2236.0,0.0,0.0,0.0,0.0,0.0,2.306422e+06,21148.0,16278.0,76380.207088,478024.0,0.0,0.0,0.0,57831.0,0.0,257639.565836,0.0,0.0,0.0,136016.480131,11107.0,70502.248076,404068.0,0.0,0.0,2.520208e+06,961232.479212,193176.386595,247.0,0.0,0.0,225.0,0.0,2016-01-11,9143.0,0.0,198123.396746,471254.197728,87383.0,0.0,0.0,353354.844840,0.0,de,0.000000,0.0,0.0,0.0,386722.377913,706.0,0.0,17340.0,49076.0,278878.649717,Sheet1,76380.20708756169,NaT
2,806.0,0.0,178895.0,0.0,231.027642,1.0,0.0,116345.872180,4963.0,0.0,0.0,0.0,2224.0,0.0,0.0,0.0,0.0,0.0,2.379034e+06,19715.0,15822.0,81636.483459,482391.0,0.0,0.0,0.0,104754.0,0.0,247711.892237,0.0,0.0,0.0,126167.255474,10048.0,61647.322276,388746.0,0.0,0.0,2.538803e+06,912773.599042,191568.477802,10249.0,0.0,0.0,231.0,0.0,2016-01-18,8568.0,0.0,186402.476444,439863.176592,85257.0,0.0,0.0,332119.415416,0.0,de,1.459669,0.0,0.0,1.0,372694.075197,813.0,0.0,16873.0,45575.0,288057.500725,Sheet1,81636.48345888322,NaT


In [28]:
# Save cleaned dataset
clean_out_csv = OUTPUTS_DIR / "cleaned_globestay.csv"
clean_out_parquet = OUTPUTS_DIR / "cleaned_globestay.parquet"

clean.to_csv(clean_out_csv, index=False)
try:
    clean.to_parquet(clean_out_parquet, index=False)
except Exception as e:
    log(f"WARN: Parquet save failed: {e}")

log(f"Saved cleaned CSV: {clean_out_csv}")
log(f"Saved cleaned Parquet (if succeeded): {clean_out_parquet}")


[2025-10-12 10:36:14] Saved cleaned CSV: c:\Users\Administrator.等闲的电脑\Downloads\831\outputs\cleaned_globestay.csv
[2025-10-12 10:36:14] Saved cleaned Parquet (if succeeded): c:\Users\Administrator.等闲的电脑\Downloads\831\outputs\cleaned_globestay.parquet


In [29]:
# Descriptive analytics — basic summaries

summary_blocks = {}

# Coverage
summary_blocks["row_counts_by_country"] = (
    clean.assign(country_std=clean["country_std"].fillna("Unknown"))
         .groupby("country_std", dropna=False)
         .size()
         .rename("rows")
         .reset_index()
)

# Time coverage
if "week_start" in clean.columns and clean["week_start"].notna().any():
    summary_blocks["time_coverage"] = pd.DataFrame({
        "min_week": [clean["week_start"].min()],
        "max_week": [clean["week_start"].max()],
        "num_weeks": [clean["week_start"].nunique()],
    })

# Spend and impressions by country
if spend_cols:
    summary_blocks["spend_by_country"] = (
        clean.groupby("country_std")[spend_cols].sum(min_count=1).reset_index()
    )
if impr_cols:
    summary_blocks["impr_by_country"] = (
        clean.groupby("country_std")[impr_cols].sum(min_count=1).reset_index()
    )

# Bookings by country
if col_map["bookings"]:
    summary_blocks["bookings_by_country"] = (
        clean.groupby("country_std")[col_map["bookings"]].sum(min_count=1).reset_index()
    )

# Show summaries
for name, df in summary_blocks.items():
    log(f"Summary: {name}")
    display(df.head(10))


[2025-10-12 10:36:14] Summary: row_counts_by_country


,country_std,rows
0,100513.28380878668,1
1,100741.42020209914,1
2,100839.71064254246,1
3,100978.54588799384,1
4,101058.81716947391,1
5,101336.20870672655,1
6,101514.0455284384,1
7,101583.09468886897,1
8,102871.75133082642,1
9,103116.96893421782,1


[2025-10-12 10:36:14] Summary: spend_by_country


,country_std,cost_email,fb_cost,cost_shop_other,olv_net_spend_eur,cinema_net_spend_eur,brandtv_net_spend_eur,print_net_spend_eur,cost_ppc_non_brand,cost_remarketing,ooh_net_spend_eur,cost_shop_tripadvisor,display_net_spend_eur,cost_ppc_brand,cost_shop_googleha,radio_net_spend_eur,drtv_net_spend_eur,cost_shop_trivago,yt_cost
0,100513.28380878668,0.0,2548.597463,1.384428e+05,2619.764704,0.0,0.000000e+00,0.0,3.385314e+06,100513.283809,0.0,5.428131e+05,0.000000,162182.644720,2.658518e+05,0.0,40233.460962,8.355099e+05,3.122101
1,100741.42020209914,0.0,25360.610338,8.944159e+05,25195.564396,0.0,1.777605e+06,0.0,3.582855e+06,100741.420202,0.0,1.299883e+06,31012.505476,68125.403445,8.920260e+05,0.0,140737.801431,8.240229e+05,25782.940130
2,100839.71064254246,0.0,101111.716146,2.634922e+05,0.000000,0.0,NaN,0.0,2.964437e+06,100839.710643,0.0,1.672450e+05,18222.415527,81134.047222,4.600805e+05,0.0,0.000000,4.264436e+05,80117.185547
3,100978.54588799384,0.0,3399.892761,7.376183e+04,13952.902100,0.0,0.000000e+00,0.0,2.138428e+06,100978.545888,0.0,2.135749e+05,106282.964111,35901.777185,3.135036e+05,0.0,49264.454486,5.142785e+05,0.000000
4,101058.81716947391,0.0,511.455753,1.616872e+05,0.000000,0.0,0.000000e+00,0.0,4.974948e+06,101058.817169,0.0,7.450030e+05,0.000000,63659.120352,5.054457e+05,0.0,87858.987955,1.246145e+06,0.000000
5,101336.20870672655,0.0,780.892795,1.011157e+06,0.000000,0.0,0.000000e+00,0.0,4.102902e+06,101336.208707,0.0,1.529472e+06,0.000000,67011.110293,1.178033e+06,0.0,0.000000,1.189065e+06,0.020064
6,101514.0455284384,0.0,26687.920220,6.412595e+05,0.000000,0.0,0.000000e+00,0.0,2.932909e+06,101514.045528,0.0,8.462506e+05,26179.672690,48534.758932,6.344003e+05,0.0,0.000000,1.082605e+06,0.000000
7,101583.09468886897,0.0,8921.146713,2.090045e+05,3147.370308,0.0,1.083700e+05,0.0,3.630993e+06,101583.094689,0.0,1.565286e+05,17977.230347,35589.784953,5.022130e+05,0.0,0.000000,8.208199e+05,0.000000
8,102871.75133082642,0.0,7945.186373,1.561728e+05,3.278410,0.0,1.484424e+05,0.0,2.884916e+06,102871.751331,0.0,1.404408e+05,17822.273926,36206.345188,4.070449e+05,0.0,0.000000,7.172410e+05,0.000000
9,103116.96893421782,0.0,228437.445270,6.342954e+05,0.000000,0.0,3.085702e+06,0.0,6.287413e+06,103116.968934,0.0,1.637352e+06,247401.310547,102779.341203,1.904418e+06,0.0,383006.622352,1.188038e+06,180311.732422


[2025-10-12 10:36:14] Summary: bookings_by_country


,country_std,totbookings
0,100513.28380878668,612230.0
1,100741.42020209914,595833.0
2,100839.71064254246,763689.0
3,100978.54588799384,584438.0
4,101058.81716947391,777596.0
5,101336.20870672655,646381.0
6,101514.0455284384,576955.0
7,101583.09468886897,629292.0
8,102871.75133082642,523488.0
9,103116.96893421782,1068608.0


In [30]:

if col_map["bookings"] and clean["week_start"].notna().any():
    plt.figure(figsize=(11, 5))
    ts = clean.dropna(subset=["week_start"]).copy()
    ts = ts.sort_values("week_start")
    sns.lineplot(
        data=ts, x="week_start", y=col_map["bookings"], hue="country_std", estimator=None
    )
    plt.title("Weekly bookings over time by country")
    plt.xlabel("Week start")
    plt.ylabel("Bookings")
    plt.tight_layout()
    fig_path = OUTPUTS_DIR / "ts_bookings_by_country.png"
    plt.savefig(fig_path, dpi=150)
    plt.show()
    log(f"Saved chart: {fig_path}")

if spend_cols and clean["week_start"].notna().any():
    ts = clean.dropna(subset=["week_start"]).copy()
    ts["total_spend"] = ts[spend_cols].sum(axis=1, min_count=1)
    plt.figure(figsize=(11, 5))
    sns.lineplot(data=ts.sort_values("week_start"), x="week_start", y="total_spend", hue="country_std", estimator=None)
    plt.title("Weekly total media spend over time by country")
    plt.xlabel("Week start")
    plt.ylabel("Total spend")
    plt.tight_layout()
    fig_path = OUTPUTS_DIR / "ts_total_spend_by_country.png"
    plt.savefig(fig_path, dpi=150)
    plt.show()
    log(f"Saved chart: {fig_path}")



In [31]:

value_cols = [c for c in clean.columns if re.match(r"(spend|sales|impr)[_].+", c)]

if value_cols:
    tidy = (
        clean[["week_start", "country_std"] + value_cols]
        .melt(id_vars=["week_start", "country_std"], var_name="metric_channel", value_name="value")
    )
    parts = tidy["metric_channel"].str.split("_", n=1, expand=True)
    tidy["metric"] = parts[0]
    tidy["channel"] = parts[1]

    # Summaries by country-channel
    channel_summary = (
        tidy.groupby(["country_std", "channel", "metric"]) ["value"].sum(min_count=1).reset_index()
            .pivot(index=["country_std", "channel"], columns="metric", values="value")
            .reset_index()
    )
    display(channel_summary.head(20))

    # Save
    channel_summary_out = OUTPUTS_DIR / "channel_summary.csv"
    channel_summary.to_csv(channel_summary_out, index=False)
    log(f"Saved channel summary: {channel_summary_out}")
else:
    log("No channel-like columns detected (spend_*, sales_*, impr_*). Skipping channel summary.")


metric,country_std,channel,sales
0,100513.28380878668,direct,283164.0
1,100513.28380878668,emk,19878.0
2,100513.28380878668,interco,3140.0
3,100513.28380878668,ppc_brand,49821.0
4,100513.28380878668,ppc_nonbrand,141452.0
5,100513.28380878668,retargeting,3696.0
6,100513.28380878668,shop_googleha,12455.0
7,100513.28380878668,shop_other,5777.0
8,100513.28380878668,shop_tripadvisor,17284.0
9,100513.28380878668,shop_trivago,41929.0


[2025-10-12 10:36:14] Saved channel summary: c:\Users\Administrator.等闲的电脑\Downloads\831\outputs\channel_summary.csv
